In [3]:
!pip install librosa

In [8]:
import os
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Çığlık ve diğer seslerin bulunduğu klasör yollarının tanımlanması
screams_folder = "C:\\Users\\Asus\\Desktop\\positive"
non_screams_folder = "C:\\Users\\Asus\\Desktop\\negative"

screams_files = [os.path.join(screams_folder, file) for file in os.listdir(screams_folder) if file.endswith('.wav')]
non_screams_files = [os.path.join(non_screams_folder, file) for file in os.listdir(non_screams_folder) if file.endswith('.wav')]

# Özellik çıkarma fonksiyonu
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, duration=3)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Veriyi toplayıp özellikleri çıkaralım
data = []
labels = []

# "positive" dosyaları için
for file in screams_files:
    features = extract_features(file)
    data.append(features)
    labels.append(1)

# "negative" dosyaları için
for file in non_screams_files:
    features = extract_features(file)
    data.append(features)
    labels.append(0)

# Veriyi eğitim ve test olarak bölme
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Özellikleri ölçeklendirme
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# SVM modeli oluşturma ve eğitim
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Tahminde bulunma ve başarı oranını kontrol etme
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9655172413793104


In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Desibel seviyesini hesaplayan fonksiyon
def calculate_dB(y, sr):
    # Sesin gücünü hesapla
    S = np.abs(librosa.stft(y))
    # Ortalama desibel seviyesini dön
    return librosa.amplitude_to_db(S, ref=np.max)

# Çığlık tespiti yapan fonksiyon
def detect_scream(audio_path, threshold=60):
    y, sr = librosa.load(audio_path, duration=5)  # 5 saniyelik bir örnek al
    dB = calculate_dB(y, sr)
    
    # dB değerlerinin ortalamasını al
    mean_dB = np.mean(dB, axis=1)
    
    # Histogram
    plt.figure(figsize=(14, 5))
    plt.hist(mean_dB, bins=30)
    plt.title('Ses Yoğunluğu Histogramı')
    plt.xlabel('Desibel (dB)')
    plt.ylabel('Frekans')
    plt.axvline(threshold, color='r', linestyle='dashed', linewidth=2)
    plt.show()

    # Zaman serisi üzerinde desibel yoğunluğu
    plt.figure(figsize=(14, 5))
    librosa.display.waveshow(y, sr=sr, alpha=0.5)
    plt.plot(librosa.times_like(mean_dB), mean_dB, color='r')
    plt.title('Zamanla Desibel Yoğunluğu')
    plt.xlabel('Zaman (s)')
    plt.ylabel('Desibel (dB)')
    plt.show()

    # Eşiği geçen desibel değerlerini tespit et
    peaks, _ = find_peaks(mean_dB, height=threshold)
    if peaks.size > 0:
        print("Çığlık sesi tespit edildi!")
    else:
        print("Çığlık sesi tespit edilmedi.")

# Test etmek için bir ses dosyası kullan
detect_scream("C:\\Users\\Asus\\Desktop\\negative\\clnsp1.wav")
